In [46]:
import pandas as pd
import os
import folium
import geohash

In [47]:
SAFECAST_DATA_PATH = os.path.join('results', 'fukushima_grouped.csv')

df = pd.read_csv(SAFECAST_DATA_PATH)

df.head()


,geohash,value
0,xnecec,29.000000
1,xnecee,37.000000
2,xnecef,32.714286
3,xneceq,40.807692
4,xnecer,29.800000


In [55]:
df.sort_values(by="value", ascending=False)

,geohash,value
469,xnskbc,1129.540824
41,xneest,330.000000
38,xneese,330.000000
35,xneem5,329.000000
40,xneesr,329.000000
50,xnemet,329.000000
42,xneeu2,328.000000
39,xneesf,328.000000
36,xneemh,327.000000
43,xneeu6,325.000000


In [48]:
ghash = "xnececzvd"
geohash.bbox(ghash)

{'s': 36.83672904968262,
 'w': 140.44896125793457,
 'n': 36.836771965026855,
 'e': 140.4490041732788}

In [49]:

def get_coordinates(geo):
    bbox = geohash.bbox(geo)
    return [[bbox["w"], bbox["n"]], [bbox["e"], bbox["n"]], [bbox["e"], bbox["s"]], [bbox["w"], bbox["s"]]]
#     return [[0, 0], [0, 90], [90, 90], [90, 0]]

In [50]:
features = []

for geo, _ in df.values:
    features.append({"type": "Feature", "id": geo, 
                     "geometry": {"type": "Polygon", "coordinates": [get_coordinates(geo)]}})
    
features

[{'type': 'Feature',
  'id': 'xnecec',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[140.438232421875, 36.837158203125],
     [140.44921875, 36.837158203125],
     [140.44921875, 36.8316650390625],
     [140.438232421875, 36.8316650390625]]]}},
 {'type': 'Feature',
  'id': 'xnecee',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[140.42724609375, 36.84814453125],
     [140.438232421875, 36.84814453125],
     [140.438232421875, 36.8426513671875],
     [140.42724609375, 36.8426513671875]]]}},
 {'type': 'Feature',
  'id': 'xnecef',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[140.438232421875, 36.8426513671875],
     [140.44921875, 36.8426513671875],
     [140.44921875, 36.837158203125],
     [140.438232421875, 36.837158203125]]]}},
 {'type': 'Feature',
  'id': 'xneceq',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[140.416259765625, 36.8646240234375],
     [140.42724609375, 36.8646240234375],
     [140.42724609375, 36.859130859375],
     [140.4162597656

In [51]:

geo_data = {"type": "FeatureCollection", "features": features}


In [52]:
m = folium.Map(location=[37, -102], zoom_start=5)
 
# Add the color for the choropleth:
m.choropleth(
 geo_data=geo_data,
 name='choropleth',
 data=df,
 columns=['geohash', 'value'],
 key_on='feature.id',
 fill_color='YlGn',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Radiation level'
)
folium.LayerControl().add_to(m)

m

In [53]:
m.save("map.html")